In [4]:
from pathlib import Path

import globox


path = Path("/path/to/annotations/") 
save_file = Path("coco.json")

annotations = globox.AnnotationSet.from_yolo_v7("Y:/Michigan_Tech_Courses/ALI_Work/Detectron/Large_Dataset/Enhanced_RUOD_yolo/ACDC/", image_folder="/")



AssertionError: Image classes.txt does not exist, unable to read the image size.

In [6]:
annotations

AnnotationSet(annotations: {})

In [8]:
import os
import json
import cv2
from tqdm import tqdm

# Directories
dataset_path = "Y:/Michigan_Tech_Courses/ALI_Work/Detectron/Large_Dataset/Enhanced_RUOD_yolo/ACDC/images"  # Change to your dataset path
split_folders = ["val"]  # Adjust based on your structure

# Initialize COCO structure
coco_json = {
    "images": [],
    "annotations": [],
    "categories": []
}

category_mapping = {}  # YOLO class_id -> COCO category_id
annotation_id = 0
image_id = 0

# Load YOLO classes
class_file = os.path.join(dataset_path, "classes.txt")  # Modify if needed
if os.path.exists(class_file):
    with open(class_file, "r") as f:
        classes = [line.strip() for line in f.readlines()]
    coco_json["categories"] = [{"id": i, "name": name} for i, name in enumerate(classes)]
    category_mapping = {i: i for i in range(len(classes))}
else:
    raise FileNotFoundError("classes.txt file is missing!")

# Process each dataset split
for split in split_folders:
    image_folder = os.path.join(dataset_path, split, "images")
    label_folder = os.path.join(dataset_path, split, "labels")

    if not os.path.exists(label_folder):
        print(f"Skipping {split}: Labels folder not found.")
        continue

    for filename in tqdm(os.listdir(image_folder), desc=f"Processing {split}"):
        if not filename.endswith((".jpg", ".png", ".jpeg")):
            continue

        # Image metadata
        img_path = os.path.join(image_folder, filename)
        img = cv2.imread(img_path)
        if img is None:
            continue  # Skip if image is unreadable
        height, width, _ = img.shape

        # Add image entry
        coco_json["images"].append({
            "id": image_id,
            "file_name": filename,
            "width": width,
            "height": height
        })

        # Read corresponding YOLO annotation
        label_path = os.path.join(label_folder, filename.replace(".jpg", ".txt").replace(".png", ".txt"))
        if not os.path.exists(label_path):
            image_id += 1
            continue

        with open(label_path, "r") as file:
            for line in file.readlines():
                values = line.strip().split()
                if len(values) != 5:
                    continue  # Skip invalid annotations

                class_id, x_center, y_center, bbox_width, bbox_height = map(float, values)
                if int(class_id) not in category_mapping:
                    continue

                # Convert from YOLO format to COCO format
                x_min = (x_center - bbox_width / 2) * width
                y_min = (y_center - bbox_height / 2) * height
                bbox_w = bbox_width * width
                bbox_h = bbox_height * height

                # COCO annotation
                coco_json["annotations"].append({
                    "id": annotation_id,
                    "image_id": image_id,
                    "category_id": category_mapping[int(class_id)],
                    "bbox": [x_min, y_min, bbox_w, bbox_h],
                    "area": bbox_w * bbox_h,
                    "iscrowd": 0
                })
                annotation_id += 1

        image_id += 1

# Save to COCO JSON file
output_path = os.path.join(dataset_path, "coco_annotations.json")
with open(output_path, "w") as f:
    json.dump(coco_json, f, indent=4)

print(f"COCO annotations saved to {output_path}")


Processing val:   0%|          | 0/1372 [00:00<?, ?it/s]

Processing val: 100%|██████████| 1372/1372 [00:27<00:00, 50.74it/s]


COCO annotations saved to Y:/Michigan_Tech_Courses/ALI_Work/Detectron/Large_Dataset/Enhanced_RUOD_yolo/ACDC/images\coco_annotations.json


In [2]:
from pycocotools.coco import COCO
coco = COCO("Y:/Michigan_Tech_Courses/ALI_Work/Detectron/Large_Dataset/Enhanced_RUOD_yolo/ACDC/coco_annotations.json")

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [5]:
import fiftyone as fo
dataset = fo.Dataset.from_dir(
    dataset_type=fo.types.COCODetectionDataset,
    data_path="Y:/Michigan_Tech_Courses/ALI_Work/Detectron/Large_Dataset/Enhanced_RUOD_yolo/ACDC/images/train/images",
    labels_path="Y:/Michigan_Tech_Courses/ALI_Work/Detectron/Large_Dataset/Enhanced_RUOD_yolo/ACDC/coco_annotations.json"
)
session = fo.launch_app(dataset)

 100% |█████████████████████| 0/0 [3.7ms elapsed, ? remaining, ? samples/s]  

Could not connect session, trying again in 10 seconds



RuntimeError: Client is not connected

In [12]:
import fiftyone as fo
session = fo.launch_app()